## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
! pip install gmaps

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,80.26,73,0,4.00,clear sky
1,1,Jamestown,US,42.0970,-79.2353,70.05,40,20,12.66,few clouds
2,2,Rypefjord,NO,70.6333,23.6667,43.27,75,75,11.50,broken clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,41.11,70,75,18.41,broken clouds
4,4,Raudeberg,NO,61.9875,5.1352,56.86,86,66,13.09,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# View columns of dataframe
# cols = city_data_df.columns.tolist()
# cols

# Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the customer for the maximum temperature preference. 
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,80.26,73,0,4.00,clear sky
1,1,Jamestown,US,42.0970,-79.2353,70.05,40,20,12.66,few clouds
11,11,Albany,US,42.6001,-73.9662,74.03,49,6,1.01,clear sky
12,12,Jiaocheng,CN,24.6774,116.1389,74.05,100,100,0.92,light rain
15,15,Cape Town,ZA,-33.9258,18.4232,66.38,45,0,3.00,clear sky


In [6]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().count()

City_ID                841
City                   841
Country                841
Lat                    841
Lng                    841
Max Temp               841
Humidity               841
Cloudiness             841
Wind Speed             841
Current Description    841
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,80.26,clear sky,19.7297,-155.0900,
1,Jamestown,US,70.05,few clouds,42.0970,-79.2353,
11,Albany,US,74.03,clear sky,42.6001,-73.9662,
12,Jiaocheng,CN,74.05,light rain,24.6774,116.1389,
15,Cape Town,ZA,66.38,clear sky,-33.9258,18.4232,
19,Mahebourg,MU,70.09,broken clouds,-20.4081,57.7000,
21,Broome,US,72.70,few clouds,42.2506,-75.8330,
24,Vallenar,CL,79.20,clear sky,-28.5708,-70.7581,
25,Kapaa,US,79.12,few clouds,22.0752,-159.3190,
26,Souillac,MU,70.18,light rain,-20.5167,57.5167,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping city")
        

Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not found... Skipping city
Hotel not 

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))